In [7]:
# !pip install langchain-groq

In [8]:
# !pip install langchain-community

In [9]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [10]:
from google.colab import userdata
import os

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

## 1. Define Chatbot Persona

In [11]:
system_prompt = """
You are an AI assistant named TripleAI.

ROLE:
- You are helpful, concise, and accurate.
- You NEVER make up information.
- If you don't know the answer, say: "I don't have enough information."

STYLE RULES:
- Use simple language in ENGLISH only.
- If explaining, break into bullet points.
- Avoid unnecessary emojis.

BOUNDARIES:
- Only answer questions related to {domain}.
- Reject unrelated questions politely.
"""

In [12]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("User Question:- {question}")
])

## 2. Define Groq Model

In [33]:
model = ChatGroq(model="llama-3.1-8b-instant",
                 temperature=0.5,
                 max_completion_tokens=800)

model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7c7d5098b620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c7d718b72c0>, model_name='llama-3.1-8b-instant', temperature=0.5, model_kwargs={'max_completion_tokens': 800}, groq_api_key=SecretStr('**********'))

## 3. Define the Structure of the Output

In [34]:
class ChatResponse(BaseModel):
  answer: str = Field(description='Final Answer to the User')
  confidence: str = Field(description='low / medium / high')

parser = PydanticOutputParser(pydantic_object=ChatResponse)

parser

PydanticOutputParser(pydantic_object=<class '__main__.ChatResponse'>)

In [35]:
# Update Prompt -> To Avoid OutputParserException: Invalid json output: There are three main types of Machine Learning:

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("User Question:- {question}"),
    # Return the format instructions for the JSON output.
    SystemMessagePromptTemplate.from_template("{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())

## 4. Build a Chain

In [36]:
chat_chain = chat_prompt | model | parser

chat_chain

ChatPromptTemplate(input_variables=['domain', 'question'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"answer": {"description": "Final Answer to the User", "title": "Answer", "type": "string"}, "confidence": {"description": "low / medium / high", "title": "Confidence", "type": "string"}}, "required": ["answer", "confidence"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['domain'], input_types={}, partial_variables={}, template='\nYou are an AI assistant named Triple

## 5. Run Chatbot

In [37]:
response = chat_chain.invoke({
    'domain':'Machine Learning',
    'question': 'Types of Machine Learning ?'
})

response

ChatResponse(answer='There are three primary types of Machine Learning: Supervised, Unsupervised, and Reinforcement Learning.', confidence='high')

In [38]:
print(response.answer)

There are three primary types of Machine Learning: Supervised, Unsupervised, and Reinforcement Learning.


In [39]:
print(response.confidence)

high


In [40]:
response = chat_chain.invoke({
    'domain':'Cooking',
    'question': 'How can i make a cake ?'
})

response

ChatResponse(answer="To make a cake, you'll need the following ingredients and steps:", confidence='high')

In [41]:
print(response.answer)

To make a cake, you'll need the following ingredients and steps:


In [42]:
print(response.confidence)

high
